<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/train_llama2_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/karpathy/llama2.c

Cloning into 'llama2.c'...
remote: Enumerating objects: 1434, done.
remote: Total 1434 (delta 0), reused 0 (delta 0), pack-reused 1434
Receiving objects: 100% (1434/1434), 1.17 MiB | 27.82 MiB/s, done.
Resolving deltas: 100% (871/871), done.


In [4]:
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 101.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
%cd llama2.c

/content/llama2.c


In [5]:
!python tinystories.py download

data/TinyStories_all_data.tar.gz: 100% 1.50G/1.50G [01:50<00:00, 14.5MiB/s]
Unpacking data/TinyStories_all_data.tar.gz...
Download done.
Number of shards: 50
Example story:
{'story': '\n\nLily and Ben are friends. They like to play in the park. One day, they see a big tree with a swing. Lily wants to try the swing. She runs to the tree and climbs on the swing.\n"Push me, Ben!" she says. Ben pushes her gently. Lily feels happy. She swings higher and higher. She laughs and shouts.\nBen watches Lily. He thinks she is cute. He wants to swing too. He waits for Lily to stop. But Lily does not stop. She swings faster and faster. She is having too much fun.\n"Can I swing too, Lily?" Ben asks. Lily does not hear him. She is too busy swinging. Ben feels sad. He walks away.\nLily swings so high that she loses her grip. She falls off the swing. She lands on the ground. She hurts her foot. She cries.\n"Ow, ow, ow!" she says. She looks for Ben. She wants him to help her. But Ben is not there. He is 

In [6]:
!python tinystories.py pretokenize


流式输出内容被截断，只能显示最后 5000 行内容。









 91% 91070/100000 [01:13<00:06, 1304.59it/s]


















 ... (more hidden) ...

















 91% 90830/100000 [01:13<00:07, 1301.14it/s]















 92% 92149/100000 [01:14<00:06, 1301.05it/s]
















 91% 91204/100000 [01:13<00:06, 1312.81it/s]


















 ... (more hidden) ...

















 91% 90961/100000 [01:13<00:06, 1296.64it/s]















 92% 92282/100000 [01:14<00:05, 1307.88it/s]
















 91% 91336/100000 [01:13<00:06, 1310.76it/s]


















 ... (more hidden) ...

















 91% 91091/100000 [01:13<00:07, 1264.26it/s]















 92% 92413/100000 [01:14<00:05, 1299.42it/s]
















 91% 91468/100000 [01:14<00:06, 1286.42it/s]


















 ... (more hidden) ...

















 91% 91221/100000 [01:13<00:06, 1274.47it/s]















 93% 92548/100000 [01:14<00:05, 1312.34it/s]
















 92% 91597/100000 [01:14<00:06, 1262.48it/s]


















 ... 

In [7]:
!python train.py


tokens per iteration will be: 131,072
breaks down as: 4 grad accum steps * 1 processes * 128 batch size * 256 max seq len
Traceback (most recent call last):
  File "/content/llama2.c/train.py", line 128, in <module>
    else torch.amp.autocast(device_type=device_type, dtype=ptdtype)
  File "/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py", line 234, in __init__
    raise RuntimeError('Current CUDA Device does not support bfloat16. Please switch dtype to float16.')
RuntimeError: Current CUDA Device does not support bfloat16. Please switch dtype to float16.


# llama2-zh.c


In [1]:
%cd /content
!git clone https://github.com/chenyangMl/llama2.c-zh.git
%cd llama2.c-zh

/content
Cloning into 'llama2.c-zh'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 122 (delta 57), reused 94 (delta 31), pack-reused 0
Receiving objects: 100% (122/122), 1.96 MiB | 1.91 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/content/llama2.c-zh


In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [5]:
# 下载中英文数据
!python dataProcess/tinystories.py download


data/TinyStories_all_data.tar.gz: 100% 1.50G/1.50G [00:16<00:00, 100MiB/s]
Unpacking data/TinyStories_all_data.tar.gz...
Download done.
Number of shards: 50
Example story:
{'story': '\n\nLily and Ben are friends. They like to play in the park. One day, they see a big tree with a swing. Lily wants to try the swing. She runs to the tree and climbs on the swing.\n"Push me, Ben!" she says. Ben pushes her gently. Lily feels happy. She swings higher and higher. She laughs and shouts.\nBen watches Lily. He thinks she is cute. He wants to swing too. He waits for Lily to stop. But Lily does not stop. She swings faster and faster. She is having too much fun.\n"Can I swing too, Lily?" Ben asks. Lily does not hear him. She is too busy swinging. Ben feels sad. He walks away.\nLily swings so high that she loses her grip. She falls off the swing. She lands on the ground. She hurts her foot. She cries.\n"Ow, ow, ow!" she says. She looks for Ben. She wants him to help her. But Ben is not there. He is g

In [7]:
# tokenizer 数据
!python dataProcess/tinystories.py pretokenize

流式输出内容被截断，只能显示最后 5000 行内容。


 93% 88375/95515 [05:04<00:15, 469.87it/s]









 92% 88471/95674 [05:07<00:12, 582.89it/s]



 93% 89076/95747 [05:49<00:11, 595.42it/s]
 79% 53500/67871 [07:00<00:43, 329.11it/s]













 90% 85623/95428 [05:21<00:19, 490.35it/s]




 93% 88454/95515 [05:04<00:12, 543.50it/s]











 91% 90380/99860 [05:44<00:12, 751.61it/s]









 93% 88534/95674 [05:07<00:12, 582.23it/s]
 79% 53534/67871 [07:00<00:44, 320.42it/s]



 93% 89140/95747 [05:49<00:12, 529.13it/s]













 90% 85704/95428 [05:21<00:18, 532.42it/s]




 93% 88532/95515 [05:05<00:11, 593.96it/s]











 91% 90458/99860 [05:44<00:13, 712.14it/s]









 93% 88604/95674 [05:07<00:12, 577.57it/s]



 93% 89205/95747 [05:49<00:11, 557.47it/s]
 79% 53598/67871 [07:00<00:38, 366.54it/s]













 90% 85829/95428 [05:21<00:13, 707.44it/s]




 93% 88597/95515 [05:05<00:11, 599.27it/s]











 91% 90578/99860 [05:45<00:11, 823.53it/s]
 79% 53637/67871 [07:00<00:38, 372

In [9]:
!python -m train.py --compile=False --eval_iters=10 --batch_size=8 --dtype=float16

Overriding: compile = False
Overriding: eval_iters = 10
Overriding: batch_size = 8
Overriding: dtype = float16
tokens per iteration will be: 8,192
breaks down as: 4 grad accum steps * 1 processes * 8 batch size * 256 max seq len
Initializing a new model from scratch
num decayed parameter tensors: 43, with 21,897,216 parameters
num non-decayed parameter tensors: 13, with 3,744 parameters
using fused AdamW: True
Created a PretokDataset with rng seed 42
Using both in chinese and english dataset
Created a PretokDataset with rng seed 42
Using both in chinese and english dataset
Created a PretokDataset with rng seed 42
Using both in chinese and english dataset
step 0: train loss 10.9602, val loss 10.9670
0 | loss 10.9506 | lr 0.000000e+00 | 3670.04ms | mfu -100.00%
1 | loss 10.9385 | lr 5.000000e-07 | 187.25ms | mfu -100.00%
2 | loss 10.9509 | lr 1.000000e-06 | 184.06ms | mfu -100.00%
3 | loss 10.8710 | lr 1.500000e-06 | 184.21ms | mfu -100.00%
4 | loss 10.7959 | lr 2.000000e-06 | 186.59ms |

## 小说数据集训练
- https://github.com/shjwudp/shu/tree/master/books


In [ ]:
!python dataProcess/tinyfictions.py download

In [ ]:
!python dataProcess/tinyfictions.py pretokenize


In [ ]:
# 训练. 修改batch_size=256, vocab_size=自定义的词表长度, 其他根据需要进行调整
#!python train.py
# 单卡
!python -m train.py --compile=False --eval_iters=10 --batch_size=8


# baby-llama2-chinese

In [ ]:
%cd /content
!git clone https://github.com/DLLXW/baby-llama2-chinese.git

/content
Cloning into 'baby-llama2-chinese'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 73 (delta 11), reused 11 (delta 11), pack-reused 47
Receiving objects: 100% (73/73), 479.85 KiB | 19.19 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [ ]:
%cd /content/baby-llama2-chinese

/content/baby-llama2-chinese


# 更多中文数据集和模型参考

- https://github.com/crownpku/awesome-chinese-nlp#corpus-%E4%B8%AD%E6%96%87%E8%AF%AD%E6%96%99
- https://github.com/brightmart/nlp_chinese_corpus
- NLP民工的乐园(百科全书): **https://github.com/fighting41love/funNLP**
- 大模型汇总： https://github.com/chenking2020/FindTheChatGPTer
- 超赞的超大规模中文语料集： https://github.com/esbatmop/MNBVC